In [1]:
from aocd import get_data

# Day 2: Red-Nosed Reports

## Part One

Each report is a list of numbers called levels that are separated by spaces.

In the example above, the reports can be found safe or unsafe by checking those rules:

- `7 6 4 2 1`: Safe because the levels are all decreasing by 1 or 2.
- `1 2 7 8 9`: Unsafe because 2 7 is an increase of 5.
- `9 7 6 2 1`: Unsafe because 6 2 is a decrease of 4.
- `1 3 2 4 5`: Unsafe because 1 3 is increasing but 3 2 is decreasing.
- `8 6 4 4 1`: Unsafe because 4 4 is neither an increase or a decrease.
- `1 3 6 7 9`: Safe because the levels are all increasing by 1, 2, or 3.
So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

In [2]:
import numpy as np

def assess(arr):
    # diffs between consecutive elements
    diffs = np.diff(arr)
    all_decreasing_safely = np.all((diffs == -1) | (diffs == -2) | (diffs == -3))
    all_increasing_safely = np.all((diffs == 1) | (diffs == 2) | (diffs == 3))

    return all_decreasing_safely or all_increasing_safely

Check against the examples given

In [3]:
example_input = """
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
"""

In [4]:
num_safe = 0
for line in example_input.strip().splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    is_safe = assess(array)
    print(f"Line {line} is{' not' if not is_safe else ''} safe")
    num_safe += is_safe

num_safe

Line 7 6 4 2 1 is safe
Line 1 2 7 8 9 is not safe
Line 9 7 6 2 1 is not safe
Line 1 3 2 4 5 is not safe
Line 8 6 4 4 1 is not safe
Line 1 3 6 7 9 is safe


np.int64(2)

We can't process this in a vectorized manner as the lines in the real data have variable lengths :(

In [5]:
data = get_data(day=2, year=2024)

In [6]:
num_safe = 0
for line in data.strip().splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    num_safe += assess(array)

In [7]:
num_safe

np.int64(639)

## Part Two

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

- `7 6 4 2 1`: Safe without removing any level.
- `1 2 7 8 9`: Unsafe regardless of which level is removed.
- `9 7 6 2 1`: Unsafe regardless of which level is removed.
- `1 3 2 4 5`: Safe by removing the second level, 3.
- `8 6 4 4 1`: Safe by removing the third level, 4.
- `1 3 6 7 9`: Safe without removing any level.

Thanks to the Problem Dampener, 4 reports are actually safe!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. How many reports are now safe?

- If there are two consective numbers or an increase-decrease combination then there's a possibilty of the issue being resolved.
- If there is too big a step between levels there is no possibility of resolving.

If an unsafe level change is found, drop it from the array and assess again.

In the below I tried to more efficienty find the solution by only checking unsafe problems where we see an

In [9]:
def can_resolve(arr):
    diffs = np.diff(arr)
    
    consecutive_same = (diffs == 0)
    for diff_idx in np.nonzero(consecutive_same)[0]:
        new_arr = np.delete(arr, diff_idx, 0)
        if assess(new_arr):
            return True

    increase_then_decrease = (diffs[:-1] > 0) & (diffs[1:] < 0)
    decrease_then_increase = (diffs[:-1] < 0) & (diffs[1:] > 0)
    for diff_idx in np.nonzero(increase_then_decrease | decrease_then_increase)[0]:

        fix = np.delete(arr, diff_idx, 0)
        if assess(fix):
            return True
        
        fix = np.delete(arr, diff_idx + 1, 0)
        if assess(fix):
            return True
        
        fix = np.delete(arr, diff_idx + 2, 0)
        if assess(fix):
            return True


    return False
        

In [8]:
def can_resolve(arr):
    # Check all possible single-element removals
    for i in range(len(arr)):
        new_arr = np.delete(arr, i, 0)
        if assess(new_arr):
            return True
    return False

In [10]:
num_safe = 0
for line in example_input.strip().splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    is_safe = assess(array)
    if not is_safe:
        is_safe = can_resolve(array)
    print(f"Line {line} is{' not' if not is_safe else ''} safe")
    num_safe += is_safe

num_safe

Line 7 6 4 2 1 is safe
Line 1 2 7 8 9 is not safe
Line 9 7 6 2 1 is not safe
Line 1 3 2 4 5 is safe
Line 8 6 4 4 1 is safe
Line 1 3 6 7 9 is safe


np.int64(4)

In [11]:
num_safe = 0
for line in data.strip().splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    is_safe = assess(array)
    if not is_safe:
        is_safe = can_resolve(array)

    num_safe += is_safe
        

In [12]:
num_safe

np.int64(674)